<a href="https://colab.research.google.com/github/37stu37/FFE/blob/master/Celular%20Automaton%20for%20Wildfires.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%%time 
!apt update
!apt upgrade
!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree 
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
!pip install descartes 

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Get:4 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://security.ubuntu.com/ubuntu bionic-security/multiverse amd64 Packages [7,064 B]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [817 kB]
Get:12 http://ppa.launchpad.net/marutter/c2d4u3.5/ubunt

In [0]:
import os
import geopandas as gpd

In [10]:
# Load the Drive helper and mount
from google.colab import drive
# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
!ls '/content/drive/My Drive/04_Cloud/01_Work/GNS/008_FFE'

buildingsFootprints.cpg  envelope.dbf  merge.mshp	 Welly_region.prj
buildingsFootprints.dbf  envelope.prj  merge.prj	 Welly_region.qpj
buildingsFootprints.prj  envelope.qpj  merge.qpj	 Welly_region.shp
buildingsFootprints.qpj  envelope.shp  merge.shp	 Welly_region.shx
buildingsFootprints.shp  envelope.shx  merge.shx
buildingsFootprints.shx  FFE.qgs.qgz   Welly_region.cpg
envelope.cpg		 merge.dbf     Welly_region.dbf


In [0]:
fuel_map = gpd.read_file('/content/drive/My Drive/04_Cloud/01_Work/GNS/008_FFE/merge.shp')
fuel_map.plot()

In [0]:
shake_map = gpd.read_file('/content/drive/My Drive/04_Cloud/01_Work/GNS/008_FFE/shake.shp')
fuel_map.plot()

In [0]:
fuel_map.crs

In [0]:
fuel_map.info()

In [31]:
fuel_map.SHAPE_Area.min()

0.00026637

In [0]:
from PIL import Image
im = Image.open('/content/drive/My Drive/04_Cloud/01_Work/GNS/008_FFE/FuelMapRaster.tif')
im.show()
import numpy
imarray = numpy.array(im)

In [0]:
import numpy as np
import pandas as pd
import imageio

# Cell States
# 0 = Clear, 1 = Fuel, 2 = Fire

prob = .6 # probability of a cell being fuel, otherwise it's clear
total_time = 300 # simulation time
terrain_size = [100,100] # size of the simulation: 10000 cells

In [0]:
# states hold the state of each cell
states = np.zeros((total_time,*terrain_size))
# initialize states by creating random fuel and clear cells
states[0] = np.random.choice([0,1],size=terrain_size,p=[1-prob,prob])
# set the middle cell on fire!!!
states[0,terrain_size[0]//2,terrain_size[1]//2] = 2

for t in range(1,total_time):
    # Make a copy of the original states
    states[t] = states[t-1].copy()

    for x in range(1,terrain_size[0]-1):
        for y in range(1,terrain_size[1]-1):

            if states[t-1,x,y] == 2: # It's on fire
                states[t,x,y] = 0 # Put it out and clear it
                
                # If there's fuel surrounding it
                # set it on fire!
                if states[t-1,x+1,y] == 1: 
                    states[t,x+1,y] = 2
                if states[t-1,x-1,y] == 1:
                    states[t,x-1,y] = 2
                if states[t-1,x,y+1] == 1:
                    states[t,x,y+1] = 2
                if states[t-1,x,y-1] == 1:
                    states[t,x,y-1] = 2

In [0]:
colored = np.zeros((total_time,*terrain_size,3),dtype=np.uint8)

# Color
for t in range(states.shape[0]):
    for x in range(states[t].shape[0]):
        for y in range(states[t].shape[1]):
            value = states[t,x,y].copy()

            if value == 0:
                colored[t,x,y] = [139,69,19] # Clear
            elif value == 1: 
                colored[t,x,y] = [0,255,0]   # Fuel
            elif value == 2: 
                colored[t,x,y] = [255,0,0]   # Burning
            
# Crop
cropped = colored[:200,1:terrain_size[0]-1,1:terrain_size[1]-1]

imageio.mimsave('./video.gif', cropped)